## 1. Latih model dasar (baseline model)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Baseline model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_split=0.1)
loss, accuracy = model.evaluate(x_test, y_test)
print(f"\nAkurasi pada data uji: {accuracy:.4f}")


Epoch 1/2
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2737 - accuracy: 0.9206 - val_loss: 0.1410 - val_accuracy: 0.9577
Epoch 2/2
313/313 [==============================] - 0s 539us/step - loss: 0.1032 - accuracy: 0.9692

Akurasi pada data uji: 0.9692


## 2. Pruning model menggunakan TensorFlow Model Optimization Toolkit

In [8]:
import tensorflow_model_optimization as tfmot

# Define pruning schedule
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=len(x_train) // 128 * 2  # 2 epochs
    )
}

# Apply pruning
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Re-compile and train
pruned_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

pruned_model.fit(x_train, y_train,
                 epochs=2,
                 batch_size=128,
                 validation_split=0.1,
                 callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])
loss, accuracy = pruned_model.evaluate(x_test, y_test)
print(f"\nAkurasi pada data uji: {accuracy:.4f}")

Epoch 1/2
422/422 [==============================] - 2s 2ms/step - loss: 0.0778 - accuracy: 0.9777 - val_loss: 0.0825 - val_accuracy: 0.9778
Epoch 2/2
313/313 [==============================] - 0s 699us/step - loss: 0.0805 - accuracy: 0.9760

Akurasi pada data uji: 0.9760


## 3. Post-Training Quantization (PTQ)

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Aktivasi quantisasi
tflite_quant_model = converter.convert()

# Simpan model hasil quantization
with open('model_pruned_quantized.tflite', 'wb') as f:
    f.write(tflite_quant_model)


INFO:tensorflow:Assets written to: /var/folders/qg/vb_4ypx91kvfzw2t50q5sb9h0000gn/T/tmppfn3v4v2/assets


INFO:tensorflow:Assets written to: /var/folders/qg/vb_4ypx91kvfzw2t50q5sb9h0000gn/T/tmppfn3v4v2/assets
2025-04-22 08:45:04.264043: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-04-22 08:45:04.264288: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-04-22 08:45:04.265287: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/qg/vb_4ypx91kvfzw2t50q5sb9h0000gn/T/tmppfn3v4v2
2025-04-22 08:45:04.267000: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-04-22 08:45:04.267012: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/qg/vb_4ypx91kvfzw2t50q5sb9h0000gn/T/tmppfn3v4v2
2025-04-22 08:45:04.276863: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-04-22 08:45:04.323860: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

## 4. Cek Ukuran Model Sebelum dan Sesudah

In [10]:
import os

# Save original model
model.save('original_model.h5')

# Ukuran file
print("Original Model Size (MB):", os.path.getsize("original_model.h5") / 1e6)
print("Pruned + Quantized Model Size (MB):", os.path.getsize("model_pruned_quantized.tflite") / 1e6)


Original Model Size (MB): 1.24836
Pruned + Quantized Model Size (MB): 0.81834


/Users/agungsetiabudi/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
